In [59]:
import random
import cnn_builder as cbuild
import config
import copy
import glob
import importlib
import lipiodol_methods as lm
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import niftiutils.masks as masks
import numpy as np
import os
import shutil
from os.path import *
%matplotlib inline
from scipy.ndimage.morphology import binary_closing, binary_opening, binary_dilation
from skimage.morphology import ball, label

import numpy as np
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import *
from keras.optimizers import Adam
from keras.utils import np_utils

In [94]:
importlib.reload(hf)
C = config.Config()

In [3]:
img_dir = "D:\\Lipiodol\\Images all"
seg_dir = "D:\\Lipiodol\\Images extracted and segmentations"
target_dir = "D:\\Lipiodol\\Data"

In [77]:
importlib.reload(config)
C = config.Config()

In [92]:
seg = glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids"))[0]

In [ ]:
lesion_id = "07"
glob.glob(join(target_dir,lesion_id,"CT24h","*"))

In [27]:
for f in glob.glob(join(target_dir,lesion_id,"CT24h","*")):
    if "5 0" not in f:
        shutil.rmtree(f)
    else:
        shutil.move()

# UNet

In [57]:
train_set = []
for seg in glob.glob(join(target_dir,"*","liver bl*.ids")):
    lesion_id = dirname(seg)[-2:]
    if exists(join(C.full_img_dir, lesion_id+"_mri.npy")):
        #train_set.append([np.load(join(C.full_img_dir, lesion_id+"_ct.npy")),
        #                  np.load(join(C.full_img_dir, lesion_id+"_mask.npy")),
        #                  lesion_id])
        continue
    
    mri_img, mri_dims = hf.nii_load(join(dirname(seg), "CT24h"), True, True)
    mask, _ = masks.get_mask(seg, ct_dims, mri_img.shape)

    mri_img = tr.apply_window(mri_img)
    mri_img -= np.amin(mri_img)
    
    np.save(join(C.full_img_dir, lesion_id+"_mri.npy"), ct_img)
    np.save(join(C.full_img_dir, lesion_id+"_mr_liver_mask.npy"), mask > 128)

In [ ]:
train_set = []
for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    lesion_id = dirname(seg)[-2:]
    if exists(join(C.full_img_dir, lesion_id+"_ct.npy")):
        #train_set.append([np.load(join(C.full_img_dir, lesion_id+"_ct.npy")),
        #                  np.load(join(C.full_img_dir, lesion_id+"_mask.npy")),
        #                  lesion_id])
        continue
    
    ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
    mask, _ = masks.get_mask(seg, ct_dims, ct_img.shape)

    ct_img = tr.apply_window(ct_img)
    ct_img -= np.amin(ct_img)
    
    np.save(join(C.full_img_dir, lesion_id+"_ct.npy"), ct_img)
    np.save(join(C.full_img_dir, lesion_id+"_mask.npy"), mask > 128)

In [91]:
importlib.reload(cbuild)
model = cbuild.build_ct_unet()
model.summary(120)

<module 'cnn_builder' from 'C:\\Users\\Clinton\\Documents\\Lipiodol\\cnn_builder.py'>

In [92]:
hist = model.fit_generator(cbuild.train_gen(), epochs=100, steps_per_epoch=50)

Epoch 1/100
100/100 [==============================] - 54s 542ms/step - loss: 0.4683 - acc: 0.8901
Epoch 2/100
100/100 [==============================] - 45s 453ms/step - loss: 0.2272 - acc: 0.9801
Epoch 3/100
100/100 [==============================] - 45s 455ms/step - loss: 0.1185 - acc: 0.9849
Epoch 4/100
100/100 [==============================] - 45s 452ms/step - loss: 0.0734 - acc: 0.9867
Epoch 5/100
100/100 [==============================] - 46s 456ms/step - loss: 0.0533 - acc: 0.9873
Epoch 6/100
100/100 [==============================] - 46s 460ms/step - loss: 0.0445 - acc: 0.9876
Epoch 7/100
100/100 [==============================] - 46s 459ms/step - loss: 0.0371 - acc: 0.9887
Epoch 8/100
100/100 [==============================] - 46s 459ms/step - loss: 0.0338 - acc: 0.9888
Epoch 9/100
100/100 [==============================] - 46s 460ms/step - loss: 0.0298 - acc: 0.9897
Epoch 10/100
100/100 [==============================] - 46s 463ms/step - loss: 0.0280 - acc: 0.9899
Epoch 11/

100/100 [==============================] - 47s 469ms/step - loss: 0.0143 - acc: 0.9937
Epoch 84/100
100/100 [==============================] - 47s 469ms/step - loss: 0.0144 - acc: 0.9937
Epoch 85/100
100/100 [==============================] - 47s 469ms/step - loss: 0.0132 - acc: 0.9942
Epoch 86/100
100/100 [==============================] - 47s 469ms/step - loss: 0.0139 - acc: 0.9939
Epoch 87/100
100/100 [==============================] - 47s 470ms/step - loss: 0.0137 - acc: 0.9940
Epoch 88/100
100/100 [==============================] - 47s 469ms/step - loss: 0.0140 - acc: 0.9939
Epoch 89/100
100/100 [==============================] - 47s 469ms/step - loss: 0.0127 - acc: 0.9944
Epoch 90/100
100/100 [==============================] - 47s 469ms/step - loss: 0.0139 - acc: 0.9939
Epoch 91/100
100/100 [==============================] - 47s 470ms/step - loss: 0.0137 - acc: 0.9940
Epoch 92/100
100/100 [==============================] - 47s 470ms/step - loss: 0.0139 - acc: 0.9939
Epoch 93/100


In [38]:
model.save(join(C.model_dir, "train17.hdf5"))

In [107]:
lesion_id = "18"
ct_img, ct_dims = hf.dcm_load(join(target_dir, lesion_id, "CT24h"), True, True)
lm.seg_liver_ct(ct_img, join(target_dir, lesion_id), ct_dims, model)

In [12]:
info = {}

for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
        
    seg_liver(ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims)
    break